<a href="https://colab.research.google.com/github/sanyamja1n/Zomato-Restaurant-Clustering/blob/main/Zomato_Clustering_and_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
from wordcloud import WordCloud

from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn import metrics
from sklearn.metrics import silhouette_samples, silhouette_score
import spacy

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering

from lightgbm import LGBMClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier, AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
import time

from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import plotly.express as px
import re
import string
!pip install contractions
import contractions
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('wordnet')
from nltk.corpus import stopwords


from sklearn.metrics import accuracy_score, confusion_matrix,roc_auc_score, classification_report, f1_score

import warnings
warnings.filterwarnings('ignore')


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.5/104.5 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 KB 9.8 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
#Google Drive Mount
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Loading the data
meta_df = pd.read_csv('/content/drive/MyDrive/Sample Data/Zomato Restaurant names and Metadata.csv')
reviews_df = pd.read_csv('/content/drive/MyDrive/Sample Data/Zomato Restaurant reviews.csv')

In [ ]:
# Dataset First Look
meta_df.head()

In [ ]:
reviews_df.head()

In [ ]:
# Dataset Rows & Columns count
meta_df.shape

In [ ]:
reviews_df.shape

In [ ]:
# Dataset Info
meta_df.info()

In [ ]:
reviews_df.info()

In [ ]:
# Dataset Duplicate Value Count
meta_df.duplicated().sum()

In [ ]:
reviews_df.duplicated().sum()

In [ ]:
#Dropping Duplicates
reviews_df.drop_duplicates(inplace = True)

In [ ]:
# Missing Values/Null Values Count
meta_df.isnull().sum() # Checking null values

In [ ]:
#Since more than 50% of the collection column data is missing so we will drop this column
meta_df.drop('Collections', axis = 1, inplace = True)

In [ ]:
reviews_df.isnull().sum()

In [ ]:
# Removing the null value
reviews_df.dropna(inplace= True)

In [ ]:
# Dataset Columns
meta_df.columns

In [ ]:
reviews_df.columns

In [ ]:
# Dataset Describe
reviews_df.describe(include = 'all').T

In [ ]:
meta_df.describe()

In [ ]:
# checking for unique values in cost
meta_df.Cost.value_counts()

In [ ]:
# Check Unique Values for each variable.
reviews_df.nunique()

In [ ]:
reviews_df['Rating'].value_counts()

In [ ]:
meta_df.nunique()